In [1]:
import $ivy.`org.apache.spark::spark-sql:3.5.5`
import $ivy.`org.apache.spark::spark-mllib:3.5.5`
import org.apache.log4j.{Level, Logger}
Logger.getLogger("org").setLevel(Level.WARN)

import $ivy.$
import $ivy.$
import org.apache.log4j.{Level, Logger}

# Spark ML для задачи оттока

Источник данных: [Kaggle](https://www.kaggle.com/sakshigoyal7/credit-card-customers)

Описания полей:

Название | Описание
:--------|:--------
CLIENTNUM | Client number. Unique identifier for the customer holding the account
Attrition_Flag | Internal event (customer activity) variable
Customer_Age | Demographic variable - Customer's Age in Years
Gender | Demographic variable - M=Male, F=Female
Dependent_count | Demographic variable - Number of dependents
Education_Level | Demographic variable - Educational Qualification of the account holder (example: high school, college graduate, etc.)
Marital_Status | Demographic variable - Married, Single, Divorced, Unknown
Income_Category | Demographic variable - Annual Income Category of the account holder (< $40K, $40K - 60K, $60K - $80K, $80K-$120K, > $120K, Unknown)
Card_Category | Product Variable - Type of Card (Blue, Silver, Gold, Platinum)
Months_on_book | Period of relationship with bank

## Загрузка и знакомство с данными

In [2]:
import org.apache.spark.sql._
import org.apache.spark.sql.functions._

val spark = SparkSession
                .builder()
                .master("local[*]")
                .appName("Spark ML Research")
                .config("spark.sql.debug.maxToStringFields", 1024)
                .config("spark.log.level", "WARN")
                .getOrCreate()

import spark.implicits._

Using Spark's default log4j profile: org/apache/spark/log4j2-defaults.properties
25/03/13 14:35:54 INFO SparkContext: Running Spark version 3.5.5
25/03/13 14:35:54 INFO SparkContext: OS info Mac OS X, 15.3.2, aarch64
25/03/13 14:35:54 INFO SparkContext: Java version 11.0.26
25/03/13 14:35:54 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable
Setting Spark log level to "WARN".


import org.apache.spark.sql._
import org.apache.spark.sql.functions._
spark: SparkSession = org.apache.spark.sql.SparkSession@107d3a1f
import spark.implicits._

In [3]:
val raw = spark
           .read
           .option("header", "true")
           .option("inferSchema", "true")
           .csv(s"../../data/BankChurners.csv")

val columns: Array[String] = raw.columns
val columnsLen: Int = columns.length
val colsToDrop = (columns.slice(columnsLen - 2, columnsLen) :+ columns.head).toIndexedSeq

val df = raw.drop(colsToDrop: _*)

df.show(5, truncate = false)

+-----------------+------------+------+---------------+---------------+--------------+---------------+-------------+--------------+------------------------+----------------------+---------------------+------------+-------------------+---------------+--------------------+---------------+--------------+-------------------+---------------------+
|Attrition_Flag   |Customer_Age|Gender|Dependent_count|Education_Level|Marital_Status|Income_Category|Card_Category|Months_on_book|Total_Relationship_Count|Months_Inactive_12_mon|Contacts_Count_12_mon|Credit_Limit|Total_Revolving_Bal|Avg_Open_To_Buy|Total_Amt_Chng_Q4_Q1|Total_Trans_Amt|Total_Trans_Ct|Total_Ct_Chng_Q4_Q1|Avg_Utilization_Ratio|
+-----------------+------------+------+---------------+---------------+--------------+---------------+-------------+--------------+------------------------+----------------------+---------------------+------------+-------------------+---------------+--------------------+---------------+--------------+--------

raw: DataFrame = [CLIENTNUM: int, Attrition_Flag: string ... 21 more fields]
columns: Array[String] = Array(
  "CLIENTNUM",
  "Attrition_Flag",
  "Customer_Age",
  "Gender",
  "Dependent_count",
  "Education_Level",
  "Marital_Status",
  "Income_Category",
  "Card_Category",
  "Months_on_book",
  "Total_Relationship_Count",
  "Months_Inactive_12_mon",
  "Contacts_Count_12_mon",
  "Credit_Limit",
  "Total_Revolving_Bal",
  "Avg_Open_To_Buy",
  "Total_Amt_Chng_Q4_Q1",
  "Total_Trans_Amt",
  "Total_Trans_Ct",
  "Total_Ct_Chng_Q4_Q1",
  "Avg_Utilization_Ratio",
  "Naive_Bayes_Classifier_Attrition_Flag_Card_Category_Contacts_Count_12_mon_Dependent_count_Education_Level_Months_Inactive_12_mon_1",
  "Naive_Bayes_Classifier_Attrition_Flag_Card_Category_Contacts_Count_12_mon_Dependent_count_Education_Level_Months_Inactive_12_mon_2"
)
columnsLen: Int = 23
colsToDrop: collection.immutable.IndexedSeq[String] = Vector(
  "Naive_Bayes_Classifier_Attrition_Flag_Card_Category_Contacts_Count_12_mon_Dep

### Определяем типы колонок

In [4]:
df.printSchema()

root
 |-- Attrition_Flag: string (nullable = true)
 |-- Customer_Age: integer (nullable = true)
 |-- Gender: string (nullable = true)
 |-- Dependent_count: integer (nullable = true)
 |-- Education_Level: string (nullable = true)
 |-- Marital_Status: string (nullable = true)
 |-- Income_Category: string (nullable = true)
 |-- Card_Category: string (nullable = true)
 |-- Months_on_book: integer (nullable = true)
 |-- Total_Relationship_Count: integer (nullable = true)
 |-- Months_Inactive_12_mon: integer (nullable = true)
 |-- Contacts_Count_12_mon: integer (nullable = true)
 |-- Credit_Limit: double (nullable = true)
 |-- Total_Revolving_Bal: integer (nullable = true)
 |-- Avg_Open_To_Buy: double (nullable = true)
 |-- Total_Amt_Chng_Q4_Q1: double (nullable = true)
 |-- Total_Trans_Amt: integer (nullable = true)
 |-- Total_Trans_Ct: integer (nullable = true)
 |-- Total_Ct_Chng_Q4_Q1: double (nullable = true)
 |-- Avg_Utilization_Ratio: double (nullable = true)



In [5]:
df.dtypes.foreach { dt => println(f"${dt._1}%25s\t${dt._2}") }

           Attrition_Flag	StringType
             Customer_Age	IntegerType
                   Gender	StringType
          Dependent_count	IntegerType
          Education_Level	StringType
           Marital_Status	StringType
          Income_Category	StringType
            Card_Category	StringType
           Months_on_book	IntegerType
 Total_Relationship_Count	IntegerType
   Months_Inactive_12_mon	IntegerType
    Contacts_Count_12_mon	IntegerType
             Credit_Limit	DoubleType
      Total_Revolving_Bal	IntegerType
          Avg_Open_To_Buy	DoubleType
     Total_Amt_Chng_Q4_Q1	DoubleType
          Total_Trans_Amt	IntegerType
           Total_Trans_Ct	IntegerType
      Total_Ct_Chng_Q4_Q1	DoubleType
    Avg_Utilization_Ratio	DoubleType


In [6]:
df.dtypes.groupBy(_._2).mapValues(_.length).foreach(println)

(StringType,6)
(IntegerType,9)
(DoubleType,5)


### Проверим числовые колонки

In [ ]:
val numericColumns = df
                      .dtypes
                      .filter(p => p._2.equals("DoubleType") || p._2.equals("IntegerType"))
                      .map(_._1)

df.select(numericColumns.map(col).toIndexedSeq: _*).summary().show()

+-------+-----------------+------------------+------------------+------------------------+----------------------+---------------------+-----------------+-------------------+-----------------+--------------------+-----------------+-----------------+-------------------+---------------------+
|summary|     Customer_Age|   Dependent_count|    Months_on_book|Total_Relationship_Count|Months_Inactive_12_mon|Contacts_Count_12_mon|     Credit_Limit|Total_Revolving_Bal|  Avg_Open_To_Buy|Total_Amt_Chng_Q4_Q1|  Total_Trans_Amt|   Total_Trans_Ct|Total_Ct_Chng_Q4_Q1|Avg_Utilization_Ratio|
+-------+-----------------+------------------+------------------+------------------------+----------------------+---------------------+-----------------+-------------------+-----------------+--------------------+-----------------+-----------------+-------------------+---------------------+
|  count|            10127|             10127|             10127|                   10127|                 10127|              

numericColumns: Array[String] = Array(
  "Customer_Age",
  "Dependent_count",
  "Months_on_book",
  "Total_Relationship_Count",
  "Months_Inactive_12_mon",
  "Contacts_Count_12_mon",
  "Credit_Limit",
  "Total_Revolving_Bal",
  "Avg_Open_To_Buy",
  "Total_Amt_Chng_Q4_Q1",
  "Total_Trans_Amt",
  "Total_Trans_Ct",
  "Total_Ct_Chng_Q4_Q1",
  "Avg_Utilization_Ratio"
)

In [58]:
import $ivy.`org.plotly-scala::plotly-almond:0.8.1`

import plotly._
import plotly.element._
import plotly.layout._
import plotly.Almond._

val ages = df
            .groupBy($"Customer_Age")
            .count()
            .orderBy($"Customer_Age")
            .collect()
            .map(a => (a(0).asInstanceOf[Int], a(1).asInstanceOf[Long]))

val x = ages.map(_._1).toSeq
val y = ages.map(_._2).toSeq

Scatter(x, y).plot()

import $ivy.$
import plotly._
import plotly.element._
import plotly.layout._
import plotly.Almond._
ages: Array[(Int, Long)] = Array(
  (26, 78L),
  (27, 32L),
  (28, 29L),
  (29, 56L),
  (30, 70L),
  (31, 91L),
  (32, 106L),
  (33, 127L),
  (34, 146L),
  (35, 184L),
  (36, 221L),
  (37, 260L),
  (38, 303L),
  (39, 333L),
  (40, 361L),
  (41, 379L),
  (42, 426L),
  (43, 473L),
  (44, 500L),
  (45, 486L),
  (46, 490L),
  (47, 479L),
  (48, 472L),
  (49, 495L),
  (50, 452L),
  (51, 398L),
  (52, 376L),
  (53, 387L),
  (54, 307L),
  (55, 279L),
  (56, 262L),
  (57, 223L),
  (58, 157L),
  (59, 157L),
  (60, 127L),
  (61, 93L),
  (62, 93L),
  (63, 65L),
...
x: Seq[Int] = WrappedArray(
  26,
  27,
  28,
  29,
  30,
  31,
  32,
  33,
  34,
  35,
  36,
  37,
  38,
  39,
  40,
  41,
  42,
  43,
  44,
  45,
  46,
  47,
  48,
  49,
  50,
  51,
  52,
  53,
  54,
  55,
  56,
  57,
  58,
  59,
  60,
  61,
  62,
  63,
...
y: Seq[Long] = WrappedArray(
  78L,
  32L,
  29L,
  56L,
  70L,
  91L,
  106L,


## Целевая колонка

In [9]:
val dft = df.withColumn("target", when($"Attrition_Flag" === "Existing Customer", 0).otherwise(1))

dft.select("Attrition_Flag", "target").show(5, truncate = false)

+-----------------+------+
|Attrition_Flag   |target|
+-----------------+------+
|Existing Customer|0     |
|Existing Customer|0     |
|Existing Customer|0     |
|Existing Customer|0     |
|Existing Customer|0     |
+-----------------+------+
only showing top 5 rows



dft: DataFrame = [Attrition_Flag: string, Customer_Age: int ... 19 more fields]

### Проверка сбалансированности данных

In [10]:
dft.groupBy("target").count().show()

+------+-----+
|target|count|
+------+-----+
|     1| 1627|
|     0| 8500|
+------+-----+



### Oversampling

In [11]:
val df1 = dft.filter($"target" === 1)
val df0 = dft.filter($"target" === 0)

val df1count = df1.count()
val df0count = df0.count()

df0count / df1count

df1: Dataset[Row] = [Attrition_Flag: string, Customer_Age: int ... 19 more fields]
df0: Dataset[Row] = [Attrition_Flag: string, Customer_Age: int ... 19 more fields]
df1count: Long = 1627L
df0count: Long = 8500L
res11_4: Long = 5L

In [12]:
val times = (df0count / df1count).toInt

times: Int = 5

### Вариант 1

In [13]:
val df1Over = df1
        .withColumn("dummy", explode(lit((1 to times).toArray)))
        .drop("dummy")

df1Over.show(10, truncate = false)

+-----------------+------------+------+---------------+---------------+--------------+---------------+-------------+--------------+------------------------+----------------------+---------------------+------------+-------------------+---------------+--------------------+---------------+--------------+-------------------+---------------------+------+
|Attrition_Flag   |Customer_Age|Gender|Dependent_count|Education_Level|Marital_Status|Income_Category|Card_Category|Months_on_book|Total_Relationship_Count|Months_Inactive_12_mon|Contacts_Count_12_mon|Credit_Limit|Total_Revolving_Bal|Avg_Open_To_Buy|Total_Amt_Chng_Q4_Q1|Total_Trans_Amt|Total_Trans_Ct|Total_Ct_Chng_Q4_Q1|Avg_Utilization_Ratio|target|
+-----------------+------------+------+---------------+---------------+--------------+---------------+-------------+--------------+------------------------+----------------------+---------------------+------------+-------------------+---------------+--------------------+---------------+---------

df1Over: DataFrame = [Attrition_Flag: string, Customer_Age: int ... 19 more fields]

### Вариант 2

In [14]:
val df1Over2 = (1 to times).toList.map(a => df1).reduce((x, y) => x.union(y))

df1Over2: Dataset[Row] = [Attrition_Flag: string, Customer_Age: int ... 19 more fields]

In [15]:
df1Over2.show(10, truncate = false)

+-----------------+------------+------+---------------+---------------+--------------+---------------+-------------+--------------+------------------------+----------------------+---------------------+------------+-------------------+---------------+--------------------+---------------+--------------+-------------------+---------------------+------+
|Attrition_Flag   |Customer_Age|Gender|Dependent_count|Education_Level|Marital_Status|Income_Category|Card_Category|Months_on_book|Total_Relationship_Count|Months_Inactive_12_mon|Contacts_Count_12_mon|Credit_Limit|Total_Revolving_Bal|Avg_Open_To_Buy|Total_Amt_Chng_Q4_Q1|Total_Trans_Amt|Total_Trans_Ct|Total_Ct_Chng_Q4_Q1|Avg_Utilization_Ratio|target|
+-----------------+------------+------+---------------+---------------+--------------+---------------+-------------+--------------+------------------------+----------------------+---------------------+------------+-------------------+---------------+--------------------+---------------+---------

In [16]:
df1Over.except(df1Over2).show

+--------------+------------+------+---------------+---------------+--------------+---------------+-------------+--------------+------------------------+----------------------+---------------------+------------+-------------------+---------------+--------------------+---------------+--------------+-------------------+---------------------+------+
|Attrition_Flag|Customer_Age|Gender|Dependent_count|Education_Level|Marital_Status|Income_Category|Card_Category|Months_on_book|Total_Relationship_Count|Months_Inactive_12_mon|Contacts_Count_12_mon|Credit_Limit|Total_Revolving_Bal|Avg_Open_To_Buy|Total_Amt_Chng_Q4_Q1|Total_Trans_Amt|Total_Trans_Ct|Total_Ct_Chng_Q4_Q1|Avg_Utilization_Ratio|target|
+--------------+------------+------+---------------+---------------+--------------+---------------+-------------+--------------+------------------------+----------------------+---------------------+------------+-------------------+---------------+--------------------+---------------+--------------+---

### Объединяем

In [17]:
val data = df0.unionAll(df1Over)
data.groupBy("target").count().show()

+------+-----+
|target|count|
+------+-----+
|     0| 8500|
|     1| 8135|
+------+-----+



data: Dataset[Row] = [Attrition_Flag: string, Customer_Age: int ... 19 more fields]

## Работа с признаками

### Проверим корреляции числовых признаков

In [18]:
val numericColumnsPairs = numericColumns.flatMap(f1 => numericColumns.map(f2 => (f1, f2)))

numericColumnsPairs: Array[(String, String)] = Array(
  ("Customer_Age", "Customer_Age"),
  ("Customer_Age", "Dependent_count"),
  ("Customer_Age", "Months_on_book"),
  ("Customer_Age", "Total_Relationship_Count"),
  ("Customer_Age", "Months_Inactive_12_mon"),
  ("Customer_Age", "Contacts_Count_12_mon"),
  ("Customer_Age", "Credit_Limit"),
  ("Customer_Age", "Total_Revolving_Bal"),
  ("Customer_Age", "Avg_Open_To_Buy"),
  ("Customer_Age", "Total_Amt_Chng_Q4_Q1"),
  ("Customer_Age", "Total_Trans_Amt"),
  ("Customer_Age", "Total_Trans_Ct"),
  ("Customer_Age", "Total_Ct_Chng_Q4_Q1"),
  ("Customer_Age", "Avg_Utilization_Ratio"),
  ("Dependent_count", "Customer_Age"),
  ("Dependent_count", "Dependent_count"),
  ("Dependent_count", "Months_on_book"),
  ("Dependent_count", "Total_Relationship_Count"),
  ("Dependent_count", "Months_Inactive_12_mon"),
  ("Dependent_count", "Contacts_Count_12_mon"),
  ("Dependent_count", "Credit_Limit"),
  ("Dependent_count", "Total_Revolving_Bal"),
  ("Dependen

In [19]:
for {
  x <- numericColumns
  y <- numericColumns
} yield (x, y)

res19: Array[(String, String)] = Array(
  ("Customer_Age", "Customer_Age"),
  ("Customer_Age", "Dependent_count"),
  ("Customer_Age", "Months_on_book"),
  ("Customer_Age", "Total_Relationship_Count"),
  ("Customer_Age", "Months_Inactive_12_mon"),
  ("Customer_Age", "Contacts_Count_12_mon"),
  ("Customer_Age", "Credit_Limit"),
  ("Customer_Age", "Total_Revolving_Bal"),
  ("Customer_Age", "Avg_Open_To_Buy"),
  ("Customer_Age", "Total_Amt_Chng_Q4_Q1"),
  ("Customer_Age", "Total_Trans_Amt"),
  ("Customer_Age", "Total_Trans_Ct"),
  ("Customer_Age", "Total_Ct_Chng_Q4_Q1"),
  ("Customer_Age", "Avg_Utilization_Ratio"),
  ("Dependent_count", "Customer_Age"),
  ("Dependent_count", "Dependent_count"),
  ("Dependent_count", "Months_on_book"),
  ("Dependent_count", "Total_Relationship_Count"),
  ("Dependent_count", "Months_Inactive_12_mon"),
  ("Dependent_count", "Contacts_Count_12_mon"),
  ("Dependent_count", "Credit_Limit"),
  ("Dependent_count", "Total_Revolving_Bal"),
  ("Dependent_count", "Avg

#### Вариант 1: DataFrameStatFunctions

In [20]:
val pairs = numericColumnsPairs
        .filter { p => !p._1.equals(p._2) }
        .map { p => if (p._1 < p._2) (p._1, p._2) else (p._2, p._1) }
        .distinct

val corr = pairs
        .map { p => (p._1, p._2, data.stat.corr(p._1, p._2)) }
        .filter(p => math.abs(p._3) > 0.7)

pairs: Array[(String, String)] = Array(
  ("Customer_Age", "Dependent_count"),
  ("Customer_Age", "Months_on_book"),
  ("Customer_Age", "Total_Relationship_Count"),
  ("Customer_Age", "Months_Inactive_12_mon"),
  ("Contacts_Count_12_mon", "Customer_Age"),
  ("Credit_Limit", "Customer_Age"),
  ("Customer_Age", "Total_Revolving_Bal"),
  ("Avg_Open_To_Buy", "Customer_Age"),
  ("Customer_Age", "Total_Amt_Chng_Q4_Q1"),
  ("Customer_Age", "Total_Trans_Amt"),
  ("Customer_Age", "Total_Trans_Ct"),
  ("Customer_Age", "Total_Ct_Chng_Q4_Q1"),
  ("Avg_Utilization_Ratio", "Customer_Age"),
  ("Dependent_count", "Months_on_book"),
  ("Dependent_count", "Total_Relationship_Count"),
  ("Dependent_count", "Months_Inactive_12_mon"),
  ("Contacts_Count_12_mon", "Dependent_count"),
  ("Credit_Limit", "Dependent_count"),
  ("Dependent_count", "Total_Revolving_Bal"),
  ("Avg_Open_To_Buy", "Dependent_count"),
  ("Dependent_count", "Total_Amt_Chng_Q4_Q1"),
  ("Dependent_count", "Total_Trans_Amt"),
  ("Dependen

In [21]:
corr.sortBy(_._3).reverse.foreach { c => println(f"${c._1}%25s${c._2}%25s\t${c._3}") }

          Avg_Open_To_Buy             Credit_Limit	0.9952040726156252
          Total_Trans_Amt           Total_Trans_Ct	0.8053901681243807
             Customer_Age           Months_on_book	0.7805047706891143


#### Вариант 2: Correlation

In [22]:
import org.apache.spark.ml.feature.VectorAssembler
import org.apache.spark.ml.stat.Correlation
import org.apache.spark.ml.linalg.Matrix
import org.apache.spark.sql.Row

val numericAssembler = new VectorAssembler()
  .setInputCols(numericColumns)
  .setOutputCol("features")

val numeric = numericAssembler.transform(data)
val Row(matrix: Matrix) = Correlation.corr(numeric, "features").head()

25/03/13 14:36:29 WARN InstanceBuilder: Failed to load implementation from:dev.ludovic.netlib.blas.JNIBLAS


import org.apache.spark.ml.feature.VectorAssembler
import org.apache.spark.ml.stat.Correlation
import org.apache.spark.ml.linalg.Matrix
import org.apache.spark.sql.Row
numericAssembler: VectorAssembler = VectorAssembler: uid=vecAssembler_dbbbbf103394, handleInvalid=error, numInputCols=14
numeric: DataFrame = [Attrition_Flag: string, Customer_Age: int ... 20 more fields]
matrix: Matrix = 1.0                    -0.13575515707704905   ... (14 total)
-0.13575515707704905   1.0                    ...
0.780504770689084      -0.11728062823959522   ...
-0.026525310066417403  -0.03266417786351196   ...
0.13116552936201348    -0.010657501150598976  ...
-0.019944405497176732  -0.02566496168277401   ...
0.016657097778483475   0.06002070339266479    ...
-0.002841536593767456  0.0018723592997836101  ...
0.016929170875654043   0.05981512203616065    ...
-0.020559135165498028  -0.04794367154504479   ...
-0.06586475313340744   0.00700430540503166    ...
-0.07815083273365253   0.025513406218718206   ...

In [23]:
val corr2 = matrix.toArray
        .zip(numericColumnsPairs)
        .map(cnn => (cnn._2._1, cnn._2._2, cnn._1))
        .filter(p => math.abs(p._3) > 0.7 && p._3 < 1.0)
        .map { p => if (p._1 < p._2) (p._1, p._2, p._3) else (p._2, p._1, p._3) }
        .distinct

corr2: Array[(String, String, Double)] = Array(
  ("Customer_Age", "Months_on_book", 0.780504770689084),
  ("Avg_Open_To_Buy", "Credit_Limit", 0.9952040726156179),
  ("Total_Trans_Amt", "Total_Trans_Ct", 0.8053901681243786)
)

In [24]:
corr2.sortBy(_._3).reverse.foreach { c => println(f"${c._1}%25s${c._2}%25s\t${c._3}") }

          Avg_Open_To_Buy             Credit_Limit	0.9952040726156179
          Total_Trans_Amt           Total_Trans_Ct	0.8053901681243786
             Customer_Age           Months_on_book	0.780504770689084


In [25]:
corr.toSet.intersect(corr2.toSet)

res25: Set[(String, String, Double)] = Set()

#### Оставляем числовые колонки с низкой кореляцией

In [26]:
val numericColumnsFinal = numericColumns.diff(corr.map(_._2))

numericColumnsFinal: Array[String] = Array(
  "Customer_Age",
  "Dependent_count",
  "Total_Relationship_Count",
  "Months_Inactive_12_mon",
  "Contacts_Count_12_mon",
  "Total_Revolving_Bal",
  "Avg_Open_To_Buy",
  "Total_Amt_Chng_Q4_Q1",
  "Total_Trans_Amt",
  "Total_Ct_Chng_Q4_Q1",
  "Avg_Utilization_Ratio"
)

### Категориальные признаки

#### Индексируем строковые колонки

In [27]:
import org.apache.spark.ml.feature.StringIndexer

val stringColumns = data
        .dtypes
        .filter(_._2.equals("StringType"))
        .map(_._1)
        .filter(!_.equals("Attrition_Flag"))

val stringColumnsIndexed = stringColumns.map(_ + "_Indexed")

val indexer = new StringIndexer()
        .setInputCols(stringColumns)
        .setOutputCols(stringColumnsIndexed)

val indexed = indexer.fit(data).transform(data)
indexed.show(5)

+-----------------+------------+------+---------------+---------------+--------------+---------------+-------------+--------------+------------------------+----------------------+---------------------+------------+-------------------+---------------+--------------------+---------------+--------------+-------------------+---------------------+------+--------------+-----------------------+----------------------+-----------------------+---------------------+
|   Attrition_Flag|Customer_Age|Gender|Dependent_count|Education_Level|Marital_Status|Income_Category|Card_Category|Months_on_book|Total_Relationship_Count|Months_Inactive_12_mon|Contacts_Count_12_mon|Credit_Limit|Total_Revolving_Bal|Avg_Open_To_Buy|Total_Amt_Chng_Q4_Q1|Total_Trans_Amt|Total_Trans_Ct|Total_Ct_Chng_Q4_Q1|Avg_Utilization_Ratio|target|Gender_Indexed|Education_Level_Indexed|Marital_Status_Indexed|Income_Category_Indexed|Card_Category_Indexed|
+-----------------+------------+------+---------------+---------------+---------

import org.apache.spark.ml.feature.StringIndexer
stringColumns: Array[String] = Array(
  "Gender",
  "Education_Level",
  "Marital_Status",
  "Income_Category",
  "Card_Category"
)
stringColumnsIndexed: Array[String] = Array(
  "Gender_Indexed",
  "Education_Level_Indexed",
  "Marital_Status_Indexed",
  "Income_Category_Indexed",
  "Card_Category_Indexed"
)
indexer: StringIndexer = strIdx_1d467ef04a06
indexed: DataFrame = [Attrition_Flag: string, Customer_Age: int ... 24 more fields]

#### Кодируем категориальные признаки

In [28]:
import org.apache.spark.ml.feature.OneHotEncoder

val catColumns = stringColumnsIndexed.map(_ + "_Coded")
    
val encoder = new OneHotEncoder()
        .setInputCols(stringColumnsIndexed)
        .setOutputCols(catColumns)

val encoderModel = encoder.fit(indexed)

import org.apache.spark.ml.feature.OneHotEncoder
catColumns: Array[String] = Array(
  "Gender_Indexed_Coded",
  "Education_Level_Indexed_Coded",
  "Marital_Status_Indexed_Coded",
  "Income_Category_Indexed_Coded",
  "Card_Category_Indexed_Coded"
)
encoder: OneHotEncoder = oneHotEncoder_45fb646fb011
encoderModel: org.apache.spark.ml.feature.OneHotEncoderModel = OneHotEncoderModel: uid=oneHotEncoder_45fb646fb011, dropLast=true, handleInvalid=error, numInputCols=5, numOutputCols=5

In [29]:
val catSizes = encoderModel.categorySizes

catSizes: Array[Int] = Array(2, 7, 4, 6, 4)

In [30]:
val encoded = encoderModel.transform(indexed)
encoded.show(5)

+-----------------+------------+------+---------------+---------------+--------------+---------------+-------------+--------------+------------------------+----------------------+---------------------+------------+-------------------+---------------+--------------------+---------------+--------------+-------------------+---------------------+------+--------------+-----------------------+----------------------+-----------------------+---------------------+--------------------+-----------------------------+----------------------------+-----------------------------+---------------------------+
|   Attrition_Flag|Customer_Age|Gender|Dependent_count|Education_Level|Marital_Status|Income_Category|Card_Category|Months_on_book|Total_Relationship_Count|Months_Inactive_12_mon|Contacts_Count_12_mon|Credit_Limit|Total_Revolving_Bal|Avg_Open_To_Buy|Total_Amt_Chng_Q4_Q1|Total_Trans_Amt|Total_Trans_Ct|Total_Ct_Chng_Q4_Q1|Avg_Utilization_Ratio|target|Gender_Indexed|Education_Level_Indexed|Marital_Stat

encoded: DataFrame = [Attrition_Flag: string, Customer_Age: int ... 29 more fields]

In [31]:
val codedColumns = stringColumnsIndexed.zip(catSizes.map(_ - 1)).flatMap(i => (1 to i._2).toList.map(a => i._1))

codedColumns: Array[String] = Array(
  "Gender_Indexed",
  "Education_Level_Indexed",
  "Education_Level_Indexed",
  "Education_Level_Indexed",
  "Education_Level_Indexed",
  "Education_Level_Indexed",
  "Education_Level_Indexed",
  "Marital_Status_Indexed",
  "Marital_Status_Indexed",
  "Marital_Status_Indexed",
  "Income_Category_Indexed",
  "Income_Category_Indexed",
  "Income_Category_Indexed",
  "Income_Category_Indexed",
  "Income_Category_Indexed",
  "Card_Category_Indexed",
  "Card_Category_Indexed",
  "Card_Category_Indexed"
)

### Собираем признаки в вектор

In [32]:
val featureColumns = numericColumnsFinal ++ catColumns

val assembler = new VectorAssembler()
  .setInputCols(featureColumns)
  .setOutputCol("features")

val assembled = assembler.transform(encoded)
assembled.show(5, truncate = false)

+-----------------+------------+------+---------------+---------------+--------------+---------------+-------------+--------------+------------------------+----------------------+---------------------+------------+-------------------+---------------+--------------------+---------------+--------------+-------------------+---------------------+------+--------------+-----------------------+----------------------+-----------------------+---------------------+--------------------+-----------------------------+----------------------------+-----------------------------+---------------------------+-----------------------------------------------------------------------------------------------------------------------------+
|Attrition_Flag   |Customer_Age|Gender|Dependent_count|Education_Level|Marital_Status|Income_Category|Card_Category|Months_on_book|Total_Relationship_Count|Months_Inactive_12_mon|Contacts_Count_12_mon|Credit_Limit|Total_Revolving_Bal|Avg_Open_To_Buy|Total_Amt_Chng_Q4_Q1|Total

featureColumns: Array[String] = Array(
  "Customer_Age",
  "Dependent_count",
  "Total_Relationship_Count",
  "Months_Inactive_12_mon",
  "Contacts_Count_12_mon",
  "Total_Revolving_Bal",
  "Avg_Open_To_Buy",
  "Total_Amt_Chng_Q4_Q1",
  "Total_Trans_Amt",
  "Total_Ct_Chng_Q4_Q1",
  "Avg_Utilization_Ratio",
  "Gender_Indexed_Coded",
  "Education_Level_Indexed_Coded",
  "Marital_Status_Indexed_Coded",
  "Income_Category_Indexed_Coded",
  "Card_Category_Indexed_Coded"
)
assembler: VectorAssembler = VectorAssembler: uid=vecAssembler_0b1810205a39, handleInvalid=error, numInputCols=16
assembled: DataFrame = [Attrition_Flag: string, Customer_Age: int ... 30 more fields]

In [33]:
val featureColumnsMap = (numericColumnsFinal ++ codedColumns).zipWithIndex.map(i => (i._2, i._1)).toMap

featureColumnsMap: Map[Int, String] = Map(
  0 -> "Customer_Age",
  5 -> "Total_Revolving_Bal",
  10 -> "Avg_Utilization_Ratio",
  24 -> "Income_Category_Indexed",
  25 -> "Income_Category_Indexed",
  14 -> "Education_Level_Indexed",
  20 -> "Marital_Status_Indexed",
  1 -> "Dependent_count",
  6 -> "Avg_Open_To_Buy",
  28 -> "Card_Category_Indexed",
  21 -> "Income_Category_Indexed",
  9 -> "Total_Ct_Chng_Q4_Q1",
  13 -> "Education_Level_Indexed",
  2 -> "Total_Relationship_Count",
  17 -> "Education_Level_Indexed",
  22 -> "Income_Category_Indexed",
  27 -> "Card_Category_Indexed",
  12 -> "Education_Level_Indexed",
  7 -> "Total_Amt_Chng_Q4_Q1",
  3 -> "Months_Inactive_12_mon",
  18 -> "Marital_Status_Indexed",
  16 -> "Education_Level_Indexed",
  11 -> "Gender_Indexed",
  26 -> "Card_Category_Indexed",
  23 -> "Income_Category_Indexed",
  8 -> "Total_Trans_Amt",
  19 -> "Marital_Status_Indexed",
  4 -> "Contacts_Count_12_mon",
  15 -> "Education_Level_Indexed"
)

In [34]:
assembled.select("features").show(5, truncate = false)

+-----------------------------------------------------------------------------------------------------------------------------+
|features                                                                                                                     |
+-----------------------------------------------------------------------------------------------------------------------------+
|(29,[0,1,2,3,4,5,6,7,8,9,10,13,18,24,26],[45.0,3.0,5.0,1.0,3.0,777.0,11914.0,1.335,1144.0,1.625,0.061,1.0,1.0,1.0,1.0])      |
|(29,[0,1,2,3,4,5,6,7,8,9,10,11,12,19,21,26],[49.0,5.0,6.0,1.0,2.0,864.0,7392.0,1.541,1291.0,3.714,0.105,1.0,1.0,1.0,1.0,1.0])|
|(29,[0,1,2,3,6,7,8,9,12,18,23,26],[51.0,3.0,4.0,1.0,3418.0,2.594,1887.0,2.333,1.0,1.0,1.0,1.0])                              |
|(29,[0,1,2,3,4,5,6,7,8,9,10,11,13,20,21,26],[40.0,4.0,3.0,4.0,1.0,2517.0,796.0,1.405,1171.0,2.333,0.76,1.0,1.0,1.0,1.0,1.0]) |
|(29,[0,1,2,3,6,7,8,9,15,18,24,26],[40.0,3.0,5.0,1.0,4716.0,2.175,816.0,2.5,1.0,1.0,1.0,1.0])           

### Нормализация

In [35]:
import org.apache.spark.ml.feature.MinMaxScaler

val scaler = new MinMaxScaler()
  .setInputCol("features")
  .setOutputCol("scaledFeatures")

val scaled = scaler.fit(assembled).transform(assembled)

scaled.select("features", "scaledFeatures").show(5, truncate = false)

+-----------------------------------------------------------------------------------------------------------------------------+------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------+
|features                                                                                                                     |scaledFeatures                                                                                                                                                                                                                                        |
+-----------------------------------------------------------------------------------------------------------------------------+---------------------------------------------------------------------------------------------------------------------------

import org.apache.spark.ml.feature.MinMaxScaler
scaler: MinMaxScaler = minMaxScal_caf0d549631e
scaled: DataFrame = [Attrition_Flag: string, Customer_Age: int ... 31 more fields]

## Feature Selection (отбор признаков)

In [36]:
import org.apache.spark.ml.feature.UnivariateFeatureSelector

val selector = new UnivariateFeatureSelector()
  .setFeatureType("continuous")
  .setLabelType("categorical")
  .setSelectionMode("percentile")
  .setSelectionThreshold(0.75)
  .setFeaturesCol("scaledFeatures")
  .setLabelCol("target")
  .setOutputCol("selectedFeatures")

val selectModel = selector.fit(scaled)

import org.apache.spark.ml.feature.UnivariateFeatureSelector
selector: UnivariateFeatureSelector = UnivariateFeatureSelector_fd762899cdef
selectModel: org.apache.spark.ml.feature.UnivariateFeatureSelectorModel = UnivariateFeatureSelectorModel: uid=UnivariateFeatureSelector_fd762899cdef, numSelectedFeatures=21

In [37]:
selectModel.selectedFeatures.sorted

res37: Array[Int] = Array(
  0,
  1,
  2,
  3,
  4,
  5,
  7,
  8,
  9,
  10,
  11,
  12,
  13,
  14,
  17,
  18,
  19,
  20,
  21,
  22,
  24
)

In [38]:
selectModel.selectedFeatures.length

res38: Int = 21

In [39]:
val selectedFeaturesMap = selectModel.selectedFeatures.map(featureColumnsMap.getOrElse(_, "")).zipWithIndex.map(i => (i._2, i._1)).toMap

selectedFeaturesMap: Map[Int, String] = Map(
  0 -> "Total_Relationship_Count",
  5 -> "Total_Trans_Amt",
  10 -> "Marital_Status_Indexed",
  14 -> "Customer_Age",
  20 -> "Marital_Status_Indexed",
  1 -> "Months_Inactive_12_mon",
  6 -> "Total_Ct_Chng_Q4_Q1",
  9 -> "Income_Category_Indexed",
  13 -> "Marital_Status_Indexed",
  2 -> "Contacts_Count_12_mon",
  17 -> "Education_Level_Indexed",
  12 -> "Dependent_count",
  7 -> "Avg_Utilization_Ratio",
  3 -> "Total_Revolving_Bal",
  18 -> "Education_Level_Indexed",
  16 -> "Income_Category_Indexed",
  11 -> "Income_Category_Indexed",
  8 -> "Gender_Indexed",
  19 -> "Education_Level_Indexed",
  4 -> "Total_Amt_Chng_Q4_Q1",
  15 -> "Education_Level_Indexed"
)

In [40]:
val dataF = selectModel.transform(scaled)

dataF.select("scaledFeatures", "selectedFeatures").show(5, truncate = false)

+------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------+-------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------+
|scaledFeatures                                                                                                                                                                                                                                        |selectedFeatures                                                                                                                                                                                                         |
+-----------------------------------------------------------------

dataF: DataFrame = [Attrition_Flag: string, Customer_Age: int ... 32 more fields]

## Моделирование

### Обучающая и тестовая выборки

In [41]:
val tt: Array[Dataset[Row]] = dataF.randomSplit(Array(0.7, 0.3))
val training = tt(0)
val test = tt(1)

println(s"training\t${training.count()}\ntest\t\t${test.count()}")

training	11614
test		5021


tt: Array[Dataset[Row]] = Array(
  [Attrition_Flag: string, Customer_Age: int ... 32 more fields],
  [Attrition_Flag: string, Customer_Age: int ... 32 more fields]
)
training: Dataset[Row] = [Attrition_Flag: string, Customer_Age: int ... 32 more fields]
test: Dataset[Row] = [Attrition_Flag: string, Customer_Age: int ... 32 more fields]

### Логистическая регрессия

In [42]:
import org.apache.spark.ml.classification.LogisticRegression

val lr = new LogisticRegression()
        .setMaxIter(1000)
        .setRegParam(0.2)
        .setElasticNetParam(0.8)
        .setFamily("binomial")
        .setFeaturesCol("selectedFeatures")
        .setLabelCol("target")

val lrModel = lr.fit(training)

println(s"Coefficients: ${lrModel.coefficients} Intercept: ${lrModel.intercept}")

Coefficients: (21,[8],[-1.828394327484724]) Intercept: 0.25308316482300813


import org.apache.spark.ml.classification.LogisticRegression
lr: LogisticRegression = logreg_d10f0f8d8308
lrModel: org.apache.spark.ml.classification.LogisticRegressionModel = LogisticRegressionModel: uid=logreg_d10f0f8d8308, numClasses=2, numFeatures=21

In [43]:
val coefficientsIndices = lrModel.coefficients.toSparse.indices.toList
coefficientsIndices.map(i => selectedFeaturesMap.getOrElse(i, ""))

coefficientsIndices: List[Int] = List(8)
res43_1: List[String] = List("Gender_Indexed")

### Training Summary

In [44]:
val trainingSummary = lrModel.binarySummary

println(s"accuracy: ${trainingSummary.accuracy}")
println(s"areaUnderROC: ${trainingSummary.areaUnderROC}")

accuracy: 0.6915791286378509
areaUnderROC: 0.7502842702770632


trainingSummary: org.apache.spark.ml.classification.BinaryLogisticRegressionTrainingSummary = org.apache.spark.ml.classification.BinaryLogisticRegressionTrainingSummaryImpl@2e4e56b7

### Проверяем модель на тестовой выборке

In [45]:
val predicted = lrModel.transform(test)

predicted.select("target", "rawPrediction", "probability", "prediction").show(10, truncate = false)

+------+--------------------------------------------+----------------------------------------+----------+
|target|rawPrediction                               |probability                             |prediction|
+------+--------------------------------------------+----------------------------------------+----------+
|0     |[-0.021210863195300772,0.021210863195300772]|[0.494697483000201,0.505302516999799]   |1.0       |
|0     |[0.19343101262143048,-0.19343101262143048]  |[0.5482075374829377,0.4517924625170623] |0.0       |
|0     |[0.1314014797019164,-0.1314014797019164]    |[0.5328031842339682,0.4671968157660318] |0.0       |
|0     |[-0.006934224666206273,0.006934224666206273]|[0.4982664507796982,0.5017335492203018] |1.0       |
|0     |[-0.025641544118123194,0.025641544118123194]|[0.49358996517712117,0.5064100348228788]|1.0       |
|0     |[-0.05124103389443063,0.05124103389443063]  |[0.4871925437216875,0.5128074562783125] |1.0       |
|0     |[0.1776774804513951,-0.177677480451395

predicted: DataFrame = [Attrition_Flag: string, Customer_Age: int ... 35 more fields]

In [46]:
import org.apache.spark.ml.evaluation.BinaryClassificationEvaluator

val evaluator = new BinaryClassificationEvaluator().setLabelCol("target")

println(s"areaUnderROC: ${evaluator.evaluate(predicted)}\n")

areaUnderROC: 0.7341037921909442



import org.apache.spark.ml.evaluation.BinaryClassificationEvaluator
evaluator: BinaryClassificationEvaluator = BinaryClassificationEvaluator: uid=binEval_14f9150b9693, metricName=areaUnderROC, numBins=1000

#### Confusion Matrix (матрица ошибок)

* True Positive (TP) - label is positive and prediction is also positive
* True Negative (TN) - label is negative and prediction is also negative
* False Positive (FP) - label is negative but prediction is positive
* False Negative (FN) - label is positive but prediction is negative

In [47]:
val tp = predicted.filter(($"target" === 1) and ($"prediction" === 1)).count()
val tn = predicted.filter(($"target" === 0) and ($"prediction" === 0)).count()
val fp = predicted.filter(($"target" === 0) and ($"prediction" === 1)).count()
val fn = predicted.filter(($"target" === 1) and ($"prediction" === 0)).count()

println(s"Confusion Matrix:\n$tp\t$fp\n$fn\t$tn\n")

Confusion Matrix:
1169	258
1353	2241



tp: Long = 1169L
tn: Long = 2241L
fp: Long = 258L
fn: Long = 1353L

#### Accuracy, Precision, Recall

* Accuracy (доля правильных ответов) = TP + TN / TP + TN + FP + FN
* Precision (точность) = TP / TP + FP
* Recall (полнота) = TP / TP + FN

In [48]:
val accuracy = (tp + tn) / (tp + tn + fp + fn).toDouble
val precision = tp / (tp + fp).toDouble
val recall = tp / (tp + fn).toDouble

println(s"Accuracy = $accuracy")
println(s"Precision = $precision")
println(s"Recall = $recall\n")

Accuracy = 0.6791475801633141
Precision = 0.8192011212333566
Recall = 0.46352101506740684



accuracy: Double = 0.6791475801633141
precision: Double = 0.8192011212333566
recall: Double = 0.46352101506740684

## Настраиваем модель (подбираем гиперпараметры)

In [49]:
import org.apache.spark.ml.tuning.{ParamGridBuilder, TrainValidationSplit}

val paramGrid = new ParamGridBuilder()
  .addGrid(lr.regParam, Array(0.01, 0.1, 0.5))
  .addGrid(lr.fitIntercept)
  .addGrid(lr.elasticNetParam, Array(0.0, 0.5, 1.0))
  .build()

val trainValidationSplit = new TrainValidationSplit()
  .setEstimator(lr)
  .setEvaluator(evaluator)
  .setEstimatorParamMaps(paramGrid)
  .setTrainRatio(0.7)
  .setParallelism(2)

val model = trainValidationSplit.fit(dataF)

import org.apache.spark.ml.tuning.{ParamGridBuilder, TrainValidationSplit}
paramGrid: Array[org.apache.spark.ml.param.ParamMap] = Array(
  {
	logreg_d10f0f8d8308-elasticNetParam: 0.0,
	logreg_d10f0f8d8308-fitIntercept: true,
	logreg_d10f0f8d8308-regParam: 0.01
},
  {
	logreg_d10f0f8d8308-elasticNetParam: 0.5,
	logreg_d10f0f8d8308-fitIntercept: true,
	logreg_d10f0f8d8308-regParam: 0.01
},
  {
	logreg_d10f0f8d8308-elasticNetParam: 1.0,
	logreg_d10f0f8d8308-fitIntercept: true,
	logreg_d10f0f8d8308-regParam: 0.01
},
  {
	logreg_d10f0f8d8308-elasticNetParam: 0.0,
	logreg_d10f0f8d8308-fitIntercept: false,
	logreg_d10f0f8d8308-regParam: 0.01
},
  {
	logreg_d10f0f8d8308-elasticNetParam: 0.5,
	logreg_d10f0f8d8308-fitIntercept: false,
	logreg_d10f0f8d8308-regParam: 0.01
},
  {
	logreg_d10f0f8d8308-elasticNetParam: 1.0,
	logreg_d10f0f8d8308-fitIntercept: false,
	logreg_d10f0f8d8308-regParam: 0.01
},
  {
	logreg_d10f0f8d8308-elasticNetParam: 0.0,
	logreg_d10f0f8d8308-fitIntercept: true,
	logreg_d1

In [50]:
model.bestModel.extractParamMap()

res50: org.apache.spark.ml.param.ParamMap = {
	logreg_d10f0f8d8308-aggregationDepth: 2,
	logreg_d10f0f8d8308-elasticNetParam: 0.0,
	logreg_d10f0f8d8308-family: binomial,
	logreg_d10f0f8d8308-featuresCol: selectedFeatures,
	logreg_d10f0f8d8308-fitIntercept: true,
	logreg_d10f0f8d8308-labelCol: target,
	logreg_d10f0f8d8308-maxBlockSizeInMB: 0.0,
	logreg_d10f0f8d8308-maxIter: 1000,
	logreg_d10f0f8d8308-predictionCol: prediction,
	logreg_d10f0f8d8308-probabilityCol: probability,
	logreg_d10f0f8d8308-rawPredictionCol: rawPrediction,
	logreg_d10f0f8d8308-regParam: 0.01,
	logreg_d10f0f8d8308-standardization: true,
	logreg_d10f0f8d8308-threshold: 0.5,
	logreg_d10f0f8d8308-tol: 1.0E-6
}

## Собираем всё вместе (Pipeline)

1. Отобрали числовые признаки: numericColumnsFinal
2. Проиндексировали строковые признаки: indexer
3. Закодировали категориальные признки: encoder
4. Собрали признаки в вектор: assembler
5. Нормализовали признаки: scaler
6. Провели отбор признаков: selector
7. Рассчитали модель: bestML 

In [51]:
import org.apache.spark.ml.Pipeline

val pipeline = new Pipeline().setStages(Array(indexer, encoder, assembler, scaler, selector, model.bestModel))

import org.apache.spark.ml.Pipeline
pipeline: Pipeline = pipeline_1689ca306aa8

In [52]:
val Array(trainingData, testData) = data.randomSplit(Array(0.7, 0.3))

val pipelineModel = pipeline.fit(trainingData)

trainingData: Dataset[Row] = [Attrition_Flag: string, Customer_Age: int ... 19 more fields]
testData: Dataset[Row] = [Attrition_Flag: string, Customer_Age: int ... 19 more fields]
pipelineModel: org.apache.spark.ml.PipelineModel = pipeline_1689ca306aa8

## Сохраняем модель

In [53]:
pipelineModel.write.overwrite().save(s"pipelineModel")